In [1]:
import pandas as pd
import os
import pyarrow.parquet as pq
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.models import resnet50, resnet18

In [2]:
# def get_batch(batchfile):
#     batch1 = pq.ParquetFile(batchfile)
#     it = batch1.iter_batches()
#     batch1 = next(it).to_pandas()
#     temp = batchfile.split('/')[-1].split('batch_')[1].split('.')[0]
#     batch1['Batch'] = temp
#     return(batch1)
#
# def get_pq(pqfile):
#     pq_df = pq.ParquetFile(pqfile)
#     it = pq_df.iter_batches()
#     pq_df = next(it).to_pandas()
#     return(pq_df)

In [3]:
sensor = pd.read_csv('/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
train_meta = pq.ParquetFile('/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train_meta.parquet')
# # train_meta = pq.ParquetFile("/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/batch_1.parquet")
it = train_meta.iter_batches()
train_meta = next(it).to_pandas()

In [4]:
train_meta.head(20)

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117
5,1,77,352,401,0.011372,1.295371
6,1,79,402,717,3.533397,2.479947
7,1,82,718,762,5.252108,0.672366
8,1,121,763,803,3.084929,2.065880
9,1,127,804,846,6.154335,1.371721


In [5]:
# i = 1
# while True:
#     try:
#         item = next(it)
#         i += 1
#     except StopIteration:
#         break
#
# print(i)

In [6]:
# print(train_meta)

In [7]:
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/'
# batchfile = path_batch + 'batch_1.parquet'
# batch1 = pq.ParquetFile(batchfile)
# it = batch1.iter_batches()
# batch1 = next(it).to_pandas()
# batch1.head(100)

In [8]:
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/'
# sensor_info = [get_batch(path_batch+'batch_' + str(i+1) + '.parquet') for i in tqdm(range(2))]
# sensor_info_df = pd.concat(sensor_info).reset_index()

In [9]:
# sensor_info_df.head()

In [10]:
# plt.figure(figsize = (20,10))
# sns.set(font_scale = 1,style = 'ticks')
# for i in range(16):
#     plt.subplot(4,4,i+1)
#     df = sensor_info_df[(sensor_info_df.event_id==sensor_info_df.event_id.unique()[i])]
#     sns.lineplot(data = df,x = 'time',y = 'charge',hue = 'auxiliary',)
#     plt.title('Batch 1, Event ID'+str(sensor_info_df.event_id.unique()[i]))
#     plt.legend(loc = 'upper right')
#     sns.despine()
# plt.tight_layout()

In [11]:
from Resnet import resnet101
import torch.nn as nn
model = resnet101(pretrained = False)
model.fc = nn.Sequential(nn.ReLU(),nn.Linear(in_features=2048, out_features=2)) # Changed FC layer for our task

In [12]:
model.load_state_dict(torch.load("./model_101.pth", weights_only=True))

<All keys matched successfully>

In [13]:
# import torch.nn as nn
# model = resnet34(pretrained = False)
# model.fc = nn.Sequential(nn.ReLU(),nn.Linear(in_features=512, out_features=2)) # Changed FC layer for our task
# # model = model.to(device)

In [14]:
# from CustomConvolution import CustomConvolution
# model = CustomConvolution()
# # model = model.to(device)

In [15]:
from sklearn.metrics import mean_squared_error
import numpy as np

def evaluation(dataloader):
    predictions = torch.tensor([], dtype=torch.float).to(device) # Tensor for prediction value appending
    actual = torch.tensor([], dtype=torch.float).to(device) # Tensor for answer value appending
    with torch.no_grad():
        model.eval()
        for data in dataloader:
            inputs, values = data['input_tensor'].float().to(device),data['label'].to(device)
            outputs = model(inputs).to(device)
            predictions = torch.cat((predictions, torch.stack([torch.argmax(o) for o in outputs])),0)
            actual = torch.cat((actual, values), 0)
    predictions = predictions.cpu().numpy()
    actual = actual.cpu().numpy()
    rmse = np.sqrt(mean_squared_error(predictions, actual))
    return rmse

In [16]:
from torch import optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'


pqfile = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train_meta.parquet'
path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/'

batch_num=8 # There are 660 batches total, and the batch number should be iterated in range(660).
lr = 1e-06
num_epochs = 1
batch_size = 2

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
# loss_function = nn.BCEWithLogitsLoss().to(device)
loss_function = nn.MSELoss().to(device)

In [17]:
from torch.utils.data import DataLoader
from Icecube_Dataloader import IceCube_Dataloader, collate_fn

ice_dataset = IceCube_Dataloader(pqfile, path_batch, batch_num)
train_dataset = ice_dataset
# proportions = [.75, .10, .15]
# lengths = [int(p * len(ice_dataset)) for p in proportions]
# lengths[-1] = len(ice_dataset) - sum(lengths[:-1])
# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(ice_dataset, lengths)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=11)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn, num_workers=11)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn, num_workers=11)

In [18]:
print(device)

cpu


In [19]:
# model(ice_dataset.__getitem__(0)['input_tensor'].to(device))

In [20]:
# print(ice_dataset.__getitem__(0)['input_tensor'].shape)
# for i in range(20):
#     print(ice_dataset.__getitem__(i)['input_tensor'].shape)

In [21]:
# train_dataloader.__getitem__(0)
# # temp = iter(train_dataloader)
# # next(temp)

In [22]:
params = {
    'num_epochs':num_epochs,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'train_dataloader':train_dataloader,
    # 'val_dataloader': val_dataloader,
    # 'test_dataloader': test_dataloader,
    'device':device,
    'num_epoch' : num_epochs
}


In [23]:
# train_dataset[0]['input_tensor'].shape

In [24]:
# train_dataset[0]['label'] # This would be 'azimuth','zenith'

In [ ]:
import gc
train_losses = []
outputs_ = []
accum_iter = 1

def train(model, params, accum_iter_):
    model.train()
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]
    # val_dataloader=params["val_dataloader"]
    # test_dataloader=params["test_dataloader"]

    # device=params["device"]
    # for epoch in range(0, num_epochs):
    #     with tqdm(train_dataloader,unit = 'batch') as tepoch:
    #         for dat in train_dataloader:
    #             tepoch.set_description(f"Epoch {epoch}")
    #             inputs, labels = dat['input_tensor'].to(device),dat['label'].to(device)
    #             optimizer.zero_grad()
    #             outputs = model(inputs).to(device)
    #             train_loss = loss_function(outputs.float(),labels.float())
    #             train_loss = train_loss.requires_grad_(True)
    #             train_loss.backward()
    #             optimizer.step()
    #             tepoch.set_postfix(loss=train_loss.item())

    aa = 0

    device=params["device"]
    for epoch in range(0, num_epochs):
        # for batch_idx, dat in tqdm(enumerate(train_dataloader)):
        for dat in tqdm(train_dataloader):
            inputs, labels = dat['input_tensor'].to(device),dat['label'].to(device)

            assert not torch.isnan(inputs).any(), "Inputs contain NaNs"
            assert not torch.isnan(labels).any(), "Labels contain NaNs"

            outputs = model(inputs).to(device)
            outputs_.append(outputs.cpu().detach().numpy())

            # if not torch.isnan(outputs).any():
            train_loss = loss_function(outputs.float(),labels.float())
            train_losses.append(train_loss.item())

            # if aa%10 == 0:
            #     train_losses.append(train_loss.item())
            if aa%100==0:
                print(f"Train loss: {train_loss.item()}")
                # gc.collect()
            # train_loss = train_loss.requires_grad_(True)

            train_loss.backward()

            # if ((batch_idx+1) % accum_iter_ == 0) or (batch_idx == len(train_dataloader)):
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            torch.cuda.empty_cache()
            # torch.cuda.empty_cache()
            aa = aa+1

    model.eval()
    gc.collect()
    # train_rmse = evaluation(train_dataloader)
    # val_rmse = evaluation(val_dataloader)

    # print(f"Train Loss: {train_rmse}")
    # print(" Train Loss: %.4f, Validation Loss: %.4f" %(train_rmse, val_rmse))
    torch.cuda.empty_cache()
    return 0


train(model, params, accum_iter)

  0%|          | 0/25000 [00:00<?, ?it/s]

Train loss: 2.8181722164154053


  0%|          | 101/25000 [00:27<1:23:47,  4.95it/s]

Train loss: 1.70978581905365


  0%|          | 109/25000 [00:29<2:05:35,  3.30it/s]

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(train_losses)

In [ ]:
from datetime import datetime

# Get current date and time
now = datetime.now()

# Format the date and time (e.g., "2025-06-07 15:30:45")
timestamp = now.strftime("%Y-%m-%d_%H:%M:%S")

In [ ]:
name = "./weights/model_101_"+timestamp+".pth"
torch.save(model.state_dict(), name)

In [ ]:
import pickle

# Save to file
name = "./results/losses_"+timestamp+".pickle"
with open(name, 'wb') as f:
    pickle.dump(train_losses, f)

In [ ]:
# torch.save(model.state_dict(), "./model_101.pth")

In [ ]:
# import pickle
#
# # Save to file
# with open('losses.pickle', 'wb') as f:
#     pickle.dump(train_losses, f)

In [ ]:
# torch.save(model, "./model_all.pth")

In [ ]:
# pqfile = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/test_meta.parquet'
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/test/'
# batch_num=661
# inference_dataset = ICECUDE_Dataset(pqfile,path_batch,batch_num,'test')
# inference_dataloader = DataLoader(inference_dataset, batch_size=1)

In [ ]:
# model.eval()
# output_lst=[]
# for dat in inference_dataloader:
#     inputs = dat['input_tensor'].to(device)
#     outputs = model(inputs).to(device)
#     outputs = outputs.cpu().detach().squeeze().numpy()
#     outputs = outputs.tolist()
#     output_lst.append(outputs)

In [ ]:
# sample_sub = pq.read_table('/kaggle/input/icecube-neutrinos-in-deep-ice/sample_submission.parquet').to_pandas()
# batch661 = pq.read_table('/kaggle/input/icecube-neutrinos-in-deep-ice/test/batch_661.parquet').to_pandas()


In [ ]:
# sample_sub

In [ ]:
# submission = pd.DataFrame(output_lst)
# submission.index = batch661.index.unique().tolist()
# submission.reset_index(inplace = True)
# submission.columns = ['event_id','azimuth','zenith']

In [ ]:
# submission